In [1]:
import numpy as np
import os

os.environ["KERAS_BACKEND"] = "torch"
import keras
keras.utils.set_random_seed(812)


In [2]:
#create synthetic time series random uniform integers
n = 100000
time = np.arange(0, n, 1)
data = np.random.randint(0, 6, n)

In [3]:
#select random timewindows of length 5 without overlapping
motif_indexes = []
k = 10000
p = 5
# available_starts = list(range(n - p))

# start = 0
# for i in range(k):
#     if not available_starts:
#         break
#     start = np.random.choice(available_starts)
#     motif_indexes.append(start)
#     available_starts = [idx for idx in available_starts if not idx in range(start - p, start + p)]

# motif_indexes = sorted(motif_indexes)

In [4]:
motif_indexes = []

start = 0
while len(motif_indexes) < k:
    if len(motif_indexes) % 2 == 0:
        start += p
    else:
        start += 2 * p
    motif_indexes.append(start)
motif_indexes

[5,
 15,
 20,
 30,
 35,
 45,
 50,
 60,
 65,
 75,
 80,
 90,
 95,
 105,
 110,
 120,
 125,
 135,
 140,
 150,
 155,
 165,
 170,
 180,
 185,
 195,
 200,
 210,
 215,
 225,
 230,
 240,
 245,
 255,
 260,
 270,
 275,
 285,
 290,
 300,
 305,
 315,
 320,
 330,
 335,
 345,
 350,
 360,
 365,
 375,
 380,
 390,
 395,
 405,
 410,
 420,
 425,
 435,
 440,
 450,
 455,
 465,
 470,
 480,
 485,
 495,
 500,
 510,
 515,
 525,
 530,
 540,
 545,
 555,
 560,
 570,
 575,
 585,
 590,
 600,
 605,
 615,
 620,
 630,
 635,
 645,
 650,
 660,
 665,
 675,
 680,
 690,
 695,
 705,
 710,
 720,
 725,
 735,
 740,
 750,
 755,
 765,
 770,
 780,
 785,
 795,
 800,
 810,
 815,
 825,
 830,
 840,
 845,
 855,
 860,
 870,
 875,
 885,
 890,
 900,
 905,
 915,
 920,
 930,
 935,
 945,
 950,
 960,
 965,
 975,
 980,
 990,
 995,
 1005,
 1010,
 1020,
 1025,
 1035,
 1040,
 1050,
 1055,
 1065,
 1070,
 1080,
 1085,
 1095,
 1100,
 1110,
 1115,
 1125,
 1130,
 1140,
 1145,
 1155,
 1160,
 1170,
 1175,
 1185,
 1190,
 1200,
 1205,
 1215,
 1220,
 1230,

In [5]:
motif_pattern = [1,3,5,3,1]
#change the values of the time series in the selected timewindows to the motif pattern
for idx in motif_indexes:
    data[idx:idx+p] = motif_pattern

data

array([3, 0, 0, ..., 0, 1, 2])

In [6]:
def split_sequence(sequence):
    X, y = list(), list()
    for end_ix in range(1,len(sequence)):
        seq_x, seq_y = sequence[:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return X, np.array(y)

In [7]:
X, y = split_sequence(motif_indexes)
for i in range(len(X)):
 print(X[i], y[i])

In [ ]:
#pad sequences putting zeros at the beginning
from keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X, padding='pre')
X

In [ ]:
#train validation test split
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#reshape input to be [samples, time steps, features]
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
#X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

#X_train.shape

In [ ]:
#train are the first 80% of the X
X_train = X[:int(0.8*len(X))]
X_test = X[int(0.8*len(X)):]
y_train = y[:int(0.8*len(y))]
y_test = y[int(0.8*len(y)):]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

X_train.shape

In [ ]:
#train lstm
from keras.models import Sequential
from keras.layers import LSTM, Dense, Masking
from keras import Input

def create_model(hp):
    model = Sequential()
    model.add(Input(shape=(X_train.shape[1], 1)))
    model.add(Masking(mask_value=0.))
    model.add(LSTM(units=hp.Int('units', min_value=10, max_value=30, step=10), activation='tanh'))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

In [ ]:
import keras_tuner as kt

tuner= kt.RandomSearch(
        create_model,
        objective='mae',
        max_trials=10,
        executions_per_trial=5
        )

tuner.search(
        x=X_train,
        y=y_train,
        epochs=20,
        validation_data=(X_test,y_test),
)

In [ ]:
model = Sequential()
model.add(Masking(mask_value=0.))
model.add(Dense(1000, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(10, activation='relu'))  # Additional hidden layer
model.add(Dense(1))

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
history = model.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test))